In [8]:
import time
import json
from jira import JIRA


options ={
    'server':'https://issues.apache.org/jira/'
}

project_name = "CXF"

jira = JIRA(options)
sess_get = jira._session.get
projects = jira.projects()


for p in projects:
    #print(p.name)
    if p.name == project_name:
        project = p
        print(project.name)
        break
            
issues = []            


keepCrawling = True
i = 0
while keepCrawling:
    tmp = jira.search_issues('project='+project.key+ ' AND status in (Resolved, Closed) AND resolution=Fixed', startAt=i, maxResults=5)
    print('.', end="")
    if (len(tmp) > 0):
        issues.extend(tmp)
        i = i + 100
        keepCrawling = False #temple limitation
    else:
        keepCrawling = False

#Jira on Apache has a limitation of 100
print('Total number of issues: '+str(len(issues)))



CXF
.Total number of issues: 5


In [7]:
import time
import datetime
from datetime import datetime, date
from collections import OrderedDict

storeIssues = []

timeFormat = "%Y-%m-%dT%H:%M:%S.000+0000"

for issue in issues:
    print('.', end="")
    exportedData = OrderedDict([])
    
    try:
        affectversion = {'affect':issue.fields.versions[0].name}
    except Exception:
        affectversion = {'affect':""}
        
   # try:
   #     allaffectversion = {'all_affect':issue.fields.versions.name}
   # except Exception:
   #     allaffectversion = {'all_affect':""}
    
    try:
        fixversion    = {'fix':issue.fields.fixVersions[0].name}
    except Exception:
        fixversion = {'fix':""}
        
    
    priority      = {'priority':issue.fields.priority.name}
    resolvedDate  = datetime.strptime(issue.fields.resolutiondate, timeFormat)
    createdDate   = datetime.strptime(issue.fields.created, timeFormat)
    fixdays       = {'time':(resolvedDate - createdDate).seconds}
    issue_type    = {'type':issue.fields.issuetype.name}
    issue_id      = {'issue_id':issue.key}
   
    #print(affectversion)
    #print(fixversion)
    #print(priority)
    #print(fixdays)
    exportedData.update(issue_id)
    exportedData.update(affectversion)
    exportedData.update(fixversion)
    exportedData.update(priority)
    exportedData.update(issue_type)
    exportedData.update(fixdays)
    
    DEV_STATUS = 'https://issues.apache.org/jira/rest/dev-status/1.0'
    _issue = 'issue/detail?issueId=%s' % issue.id
    _args = 'applicationType=fecru&dataType=repository&_=%s' % int(time.time())
    req_url = '%s/%s&%s' % (DEV_STATUS, _issue, _args)
    response = sess_get(req_url)
    raw_data = json.loads(response.content.decode('utf-8'))
    #print(issue)
    #print(raw_data)
    try:
        hasCommit = True
        commits = raw_data['detail'][0]['repositories'][0]['commits']
        #storeIssues.append(response.content.decode('utf-8'))
    except IndexError:
        hasCommit = False
    if hasCommit:
        commitList = []
        for commit in commits:
            #print(req)
            #print(issue.id) 
            patches = []
            #print('%s\n%s\n\n' % (req['displayId'], req['files']))
            for file in commit['files']:
                patches.append({'filename':file['path']})
            commitList.append({'files':patches})
            #print(patches)
        exportedData.update({'commits':commitList})
        storeIssues.append(exportedData)
    

            
with open(project_name+'_data.json', 'w') as outfile:
    json.dump(storeIssues, outfile)
    
#print(issues)

#issue = jira.issue('JRA-9')
#print(issue.fields.project.key)             # 'JRA'
#print(issue.fields.issuetype.name)          # 'New Feature'
#print(issue.fields.reporter.displayName)  # 'Mike Cannon-Brookes [Atlassian]'

.